# Google colab initialization

For Google colab uncomment these lines and run them to access your drive or try the second way (not tested)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
#
# sys.path.insert(1, r'/content/drive/My Drive/')

In [2]:
# Other try
# !git clone https://github.com/Alexanderstaehle/OM_project

In [3]:
# import sys
# sys.path.append("OM_project")

In [4]:
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

from utils import ml_utils, visualization, data_loading, tf_models

In [5]:
data_loading.initialize_env()
sns.set_theme()
color_map = sns.color_palette(as_cmap=True)
ml_utils.check_tpu_gpu()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of accelerators:  1


2022-02-20 17:33:37.358311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-20 17:33:37.383737: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-20 17:33:37.383936: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-20 17:33:37.414919: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Different batch sizes with fixed learning rate

## without sharpness aware minimization

### SGD with Momemtum

In [6]:
models_dict_fixed = {}
#batch_sizes = [32, 64, 128, 256, 512, 1024]
batch_sizes = [32]
lr = 0.001
training_epochs = 10

In [ ]:
# test of SAM model 

batch_size = 32

train, validation = data_loading.load_batched_and_resized_dataset(dataset_name='MNIST', batch_size=batch_size,
                                                                 img_size=32)

#optimizer = keras.optimizers.SGD(learning_rate=lr)
optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
#base_model = tf_models.build_simple_dense_model(train)
base_model = tf_models.build_simple_cnn(train)
base_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
train_callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10,
            restore_best_weights=True
        ),
        # tf.keras.callbacks.ReduceLROnPlateau(
        #     monitor="val_loss", factor=0.5,
        #     patience=3, verbose=1
        # )
    ]
model = tf_models.SAMModel(base_model, adaptive = True, rho = 2.0)    
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

models_dict_fixed[batch_size] = ml_utils.train_model(model, train, validation, epochs=1,
                                                         extra_callbacks=train_callbacks, verbose=1)

In [7]:
filename_bs = lambda bs: ml_utils.path_from_filename(f'model_fixed_lr_diff_bs_{bs}', format_ = "tf")

In [8]:
# Need sparse categorical crossentropy since our labels are in form of integers not vectors
for batch_size in batch_sizes:
    # Read training data
    train, validation = data_loading.load_batched_and_resized_dataset(dataset_name='MNIST', batch_size=batch_size,
                                                                      img_size=32)
    
    optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
    
    base_model = tf_models.build_simple_cnn(train)
    base_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model = tf_models.SAMModel(base_model, adaptive = True, rho = 2.0)    
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    train_callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filename_bs(batch_size),
            monitor='val_loss',
            mode='min',
            verbose=1,
            save_best_only=True,
            save_weights_only=True
        )
        # tf.keras.callbacks.ReduceLROnPlateau(
        #     monitor="val_loss", factor=0.5,
        #     patience=3, verbose=1
        # )
    ]
    models_dict_fixed[batch_size] = ml_utils.train_model(model, train, validation, epochs=10,
                                                         extra_callbacks=train_callbacks, verbose=1)

Epoch 1/10


2022-02-20 17:34:18.526181: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2022-02-20 17:34:19.014288: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1499/1500 [============================>.] - ETA: 0s - loss: 0.8057 - accuracy: 0.7262
Epoch 1: val_loss improved from inf to 0.24667, saving model to tmp/model_fixed_lr_diff_bs_32.tf
1500/1500 [==============================] - 7s 4ms/step - loss: 0.8054 - accuracy: 0.7263 - val_loss: 0.2467 - val_accuracy: 0.9347
Epoch 2/10
1494/1500 [============================>.] - ETA: 0s - loss: 0.2559 - accuracy: 0.9090
Epoch 2: val_loss improved from 0.24667 to 0.15793, saving model to tmp/model_fixed_lr_diff_bs_32.tf
1500/1500 [==============================] - 10s 6ms/step - loss: 0.2557 - accuracy: 0.9090 - val_loss: 0.1579 - val_accuracy: 0.9585
Epoch 3/10
1499/1500 [============================>.] - ETA: 0s - loss: 0.1969 - accuracy: 0.9277
Epoch 3: val_loss improved from 0.15793 to 0.12920, saving model to tmp/model_fixed_lr_diff_bs_32.tf
1500/1500 [==============================] - 13s 9ms/step - loss: 0.1969 - accuracy: 0.9277 - val_loss: 0.1292 - val_accuracy: 0.9655
Epoch 4/10
1495/1

In [ ]:
fixed_lr_state_filename = 'model_fixed_lr_diff_bs_state'

In [ ]:
ml_utils.save_model_state(models_dict_fixed, fixed_lr_state_filename)

In [ ]:
visualization.plot_loss_by_param(models_dict_fixed, 'batch size with fixed learning rate', 'fixed_lr_diff_bs')


#### sharpness

In [9]:
optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

model = tf_models.build_simple_cnn_sam(train, optimizer)

In [10]:
models_by_batch_size_fixed_lr = model.load_weights(filename_bs("32"))

In [ ]:
ml_utils.load_model_state(fixed_lr_filename)

In [11]:
visualization.get_sharpness(model.base_model, train)

1500/1500 [==============================] - 3s 2ms/step - loss: 0.0652 - accuracy: 0.9827


1500/1500 [==============================] - 3s 2ms/step - loss: 0.0652 - accuracy: 0.9827
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        59786     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.52045D-02    |proj g|=  1.47881D-02
1500/1500 [==============================] - 3s 2ms/step - loss: 1.6081 - accuracy: 0.5088

At iterate    1    f= -1.60812D+00    |proj g|=  2.85367D-02
  ys=-3.182E+03  -gs= 1.657E+02 BFGS update SKIPPED
1500/1500 [==============================] - 3s 2ms/step - loss: 7.3744 - accuracy: 0.4440

At iterate    2    f= -7.37442D+00    |proj g|=  2.95762D-02
  ys=-1.628E+04  -gs= 2.063E+03 BFGS update SKIPPED
1500/1500 [==============================] - 3s 2ms/step - loss: 12.9446 - accuracy: 0.3739

At iterate    3    f= -1.29446D+01    |proj g|=  2.34495D-02
  ys=-1.048E+04  -gs= 3.084E+03 BFGS update SKIPPED
1500/1500 [==============================] - 3s 2ms/step - loss:

1828.6382814730484


At iterate   10    f= -1.95439D+01    |proj g|=  2.22052D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
59786     10     11  80715     9 58386   2.221D-02  -1.954D+01
  F =  -19.543941497802734     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


- loss curve
- val loss curve
- final train and val loss (best) 
- sharpness of the minimizers 
- distance from initial weights

- training time
- training time per epoch 
- epochs needed to converge 
- parallelization 
- training time times sharpness

### ADAM

## with sharpness aware minimization

### SGD with Momentum + SAM 

### SGD with Momentum + ASAM

### ADAM + SAM 

### ADAM + ASAM

# Different batch sizes with linear increasing learning rate

In [ ]:
learning_rates = [0.001, 0.002, 0.004, 0.008, 0.016, 0.032]